In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
unzip_data("10_food_classes_10_percent.zip")

train_dir_10_percent = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
import tensorflow as tf
IMG_SIZE = (224,224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir_10_percent,
                                                                            label_mode='categorical',
                                                                            image_size = IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Sequential

data_augmentation = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomRotation(0.2)
],name='data_augmentation')

input_shape = (224,224,3)

base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name='input_layer')
x = data_augmentation(inputs)
x = base_model(x,training=False)
x = layers.GlobalAveragePooling2D(name='gloabl_average_pooling_layer')(x)
outputs = layers.Dense(10,activation='softmax',name='output_layer')(x)

model2 = tf.keras.Model(inputs, outputs)

model2.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])

In [ ]:
checkpoint_path = "ten_percent_model_checkpoints_weights/checkpoint.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               save_weights_only = True,
                                               save_best_only = False,
                                               save_freq='epoch',
                                               verbose=1)

In [ ]:
initial_epochs = 5
history_10_percent_data_aug = model2.fit(train_data_10_percent,
                                         epochs = initial_epochs,
                                         validation_data = test_data,
                                         validation_steps = int(0.25*len(test_data)),
                                         callbacks = [create_tensorboard_callback("transfer_learning","10_percent_data_aug"),
                                                      checkpoint_callback])

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-10]:
  layer.trainable = False

model2.compile(loss='categorical_crossentropy',
               optimizer=tf.keras.optimizers.Adam(lr = 0.0001),
               metrics=['accuracy'])

In [ ]:
fine_tune_epochs = initial_epochs + 5

history_fine_10_percent_data_aug = model2.fit(train_data_10_percent,
                                              epochs=fine_tune_epochs,
                                              validation_data=test_data,
                                              initial_epoch=history_10_percent_data_aug.epoch[-1],
                                              validation_steps=int(0.25*len(test_data)),
                                              callbacks = [create_tensorboard_callback("transfer_learning", "10_percent_fne_tune_last_10")])